In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import airbase
import pandas as pd
import matplotlib.pyplot as plt
import fastparquet as fp
import geopandas as gpd
from shapely.geometry import Point
import contextily as ctx
import plotly.express as px
from tqdm import tqdm

In [43]:
country_code_to_name = {
    'SI': 'Slovenia',
    'DE': 'Germany',
    'IS': 'Iceland',
    'XK': 'Kosovo',
    'CH': 'Switzerland',
    'AT': 'Austria',
    'DK': 'Denmark',
    'AL': 'Albania',
    'HR': 'Croatia',
    'LU': 'Luxembourg',
    'SK': 'Slovakia',
    'PL': 'Poland',
    'GB': 'United Kingdom',
    'IE': 'Ireland',
    'BA': 'Bosnia and Herzegovina',
    'BE': 'Belgium',
    'TR': 'Turkey',
    'LT': 'Lithuania',
    'IT': 'Italy',
    'ES': 'Spain',
    'MT': 'Malta',
    'RO': 'Romania',
    'BG': 'Bulgaria',
    'RS': 'Serbia',
    'NL': 'Netherlands',
    'CZ': 'Czech Republic',
    'PT': 'Portugal',
    'MK': 'North Macedonia',
    'SE': 'Sweden',
    'ME': 'Montenegro',
    'FR': 'France',
    'NO': 'Norway',
    'LV': 'Latvia',
    'AD': 'Andorra',
    'CY': 'Cyprus',
    'HU': 'Hungary',
    'GR': 'Greece',
    'EE': 'Estonia',
    'FI': 'Finland',
}

In [4]:
DATA_PATH = '../data/air_quality/'

In [5]:
def create_full_df(year: int = None) -> pd.DataFrame:
    """Create a full DataFrame with all countries' data.

    Args:
        year (int, optional): used to filter out data before a specific year. Defaults to None.

    Returns:
        pd.DataFrame: merged DataFrame with data and metadata.
    """
    print(f"Reading data...")
    
    client = airbase.AirbaseClient()
    countries = client.countries
    df = pd.DataFrame()
    
    countries = ['IT']
    
    for country in countries:
        print(f"Reading data for {country}")        
        try:
            df_country = fp.ParquetFile(f"{DATA_PATH}{country}").to_pandas()
            
            if year:
                df_country = df_country[df_country['Start'] > f"{year}-01-01"]
            
            df_country = df_country.dropna(subset=['Value'])
            df_country = df_country[(df_country['Value'] >= 0) & (df_country['Value'] <= 1000)]
                        
            display(df_country.head())
            print(df_country['AggType'].unique())
            print(df_country['Unit'].unique())
            print(len(df_country[df_country['AggType'] == 'hour']))
            print(len(df_country[df_country['AggType'] == 'day']))
            
            df = pd.concat([df, df_country], ignore_index=True)
        except FileNotFoundError:
            print(f"File not found for {country}. Skipping.")
    
    df_metadata = pd.read_csv(f"{DATA_PATH}/metadata.csv", sep=",", low_memory=False)
    print("Data read.")
    
    df_metadata_filtered = df_metadata[['Sampling Point Id', 'Latitude', 'Longitude']]
    df['Samplingpoint'] = df['Samplingpoint'].astype(str).str.split('/').str[1]
    print(len(df))
    df_merged = df[['Samplingpoint', 'Pollutant', 'Unit', 'Start', 'Value']].merge(df_metadata_filtered, left_on='Samplingpoint', right_on='Sampling Point Id')
    print(len(df_merged))
    df_merged['Start'] = df_merged['Start'].dt.date
    df_merged = df_merged.groupby(['Samplingpoint', 'Pollutant', 'Start', 'Latitude', 'Longitude'], as_index=False).agg({'Value': 'mean'})    
    return df_merged

In [ ]:
def aggregate_data(country: str) -> pd.DataFrame:
    """Compute the average value of each pollutant at each sampling point for each date.

    Args:
        country (str): Country code to filter data.

    Returns:
        pd.DataFrame: Aggregated DataFrame.
    """
    print(f"Reading data for {country}")        
    df = fp.ParquetFile(f"{DATA_PATH}{country}").to_pandas()  
    start_length = len(df)  
    df = df.dropna(subset=['Value'])
    df = df[(df['Value'] >= 0)]
    df['Samplingpoint'] = df['Samplingpoint'].astype(str).str.split('/').str[1]
    
    assert df.groupby('Pollutant')['Unit'].nunique().max() == 1, f"Multiple units for a single pollutant"
    assert set(df['AggType'].unique()) <= {'hour', 'day'}, f"Unexpected AggType values: {df['AggType'].unique()}"
    
    df_metadata = pd.read_csv(f"{DATA_PATH}/metadata.csv", sep=",", low_memory=False)
    print("Data read.")
    
    df_metadata_filtered = df_metadata[['Sampling Point Id', 'Latitude', 'Longitude']]
    df_merged = df[['Samplingpoint', 'Pollutant', 'Unit', 'Start', 'Value']].merge(df_metadata_filtered, left_on='Samplingpoint', right_on='Sampling Point Id')
        
    df_merged['Start'] = df_merged['Start'].dt.date
    df_merged = df_merged.groupby(
        ['Samplingpoint', 'Pollutant', 'Start', 'Latitude', 'Longitude'],
        as_index=False
    ).agg({
        'Value': 'mean',
        'Unit': 'first'
    })    
    
    print(f"Start length: {start_length} - End length: {len(df_merged)}")
    df_merged.to_csv(f"../data/aggregated_air_quality/aggregated_{country}.csv", index=False)  
    return df_merged

In [31]:
def aggregate_data_monthly(country: str) -> pd.DataFrame:
    """Compute the average value of each pollutant at each sampling point for each month.

    Args:
        country (str): Country code to filter data.

    Returns:
        pd.DataFrame: Aggregated DataFrame.
    """
    print(f"Reading data for {country}")        
    df = pd.read_csv(f"../data/aggregated_air_quality/{country}.csv", sep=",", low_memory=False)
        
    start_length = len(df)  
    df = df.dropna(subset=['Value'])
    df = df[(df['Value'] >= 0)]
    
    # Convert to datetime and extract year-month
    df['Date'] = pd.to_datetime(df['Date'])
    df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()

    # Group by month instead of full date
    df_aggregated = df.groupby(
        ['Samplingpoint', 'PollutantCode', 'PollutantName', 'Month', 'Latitude', 'Longitude'],
        as_index=False
    ).agg({
        'Value': 'mean',
        'Unit': 'first'
    })
    
    print(f"Start length: {start_length} - End length: {len(df_aggregated)}")
    df_aggregated.to_csv(f"../data/aggregated_air_quality_monthly/{country}.csv", index=False)  
    return df_aggregated

In [32]:
from tqdm import tqdm

client = airbase.AirbaseClient()
countries = client.countries

for country in tqdm(countries):
    print(f"Aggregating data for {country}")
    df = aggregate_data_monthly(country)

  0%|          | 0/39 [00:00<?, ?it/s]

Aggregating data for SI
Reading data for SI


  3%|▎         | 1/39 [00:00<00:13,  2.73it/s]

Start length: 170933 - End length: 5699
Aggregating data for DE
Reading data for DE
Start length: 8061739 - End length: 269887


  5%|▌         | 2/39 [00:13<04:43,  7.67s/it]

...

Start length: 411743 - End length: 14524
Aggregating data for EE
Reading data for EE


 97%|█████████▋| 38/39 [01:37<00:00,  1.20it/s]

Start length: 161927 - End length: 5398
Aggregating data for FI
Reading data for FI


100%|██████████| 39/39 [01:38<00:00,  2.52s/it]

Start length: 562340 - End length: 18929


In [35]:
from tqdm import tqdm

client = airbase.AirbaseClient()
countries = client.countries

# convert csv to json
for country in tqdm(countries):
    print(f"Converting data for {country}")
    df = pd.read_csv(f"../data/aggregated_air_quality_monthly/{country}.csv", sep=",", low_memory=False)
    
    # Convert to JSON
    df.to_json(f"../data/aggregated_air_quality_monthly_json/{country}.json", orient='records', lines=True)

  0%|          | 0/39 [00:00<?, ?it/s]

Converting data for SI
Converting data for DE


  5%|▌         | 2/39 [00:00<00:16,  2.26it/s]

Converting data for IS
Converting data for XK
Converting data for CH
Converting data for AT


 28%|██▊       | 11/39 [00:01<00:02, 11.08it/s]

...

 77%|███████▋  | 30/39 [00:04<00:01,  8.28it/s]

Converting data for FR


 95%|█████████▍| 37/39 [00:05<00:00,  9.69it/s]

Converting data for NO
Converting data for LV
Converting data for AD
Converting data for CY
Converting data for HU
Converting data for GR
Converting data for EE
Converting data for FI


100%|██████████| 39/39 [00:05<00:00,  7.01it/s]


In [44]:
# convert to single json

df = pd.DataFrame()
for country in tqdm(countries):
    print(f"Converting data for {country}")
    df_country = pd.read_csv(f"../data/aggregated_air_quality_monthly/{country}.csv", sep=",", low_memory=False)
    
    # Convert to JSON
    df_country['Country'] = country_code_to_name[country]
    df = pd.concat([df, df_country], ignore_index=True)
df.to_json(f"../data/aggregated_air_quality_monthly_json/all.json", orient='records', lines=True)
df = pd.read_json(f"../data/aggregated_air_quality_monthly_json/all.json", lines=True)

df.head()

  0%|          | 0/39 [00:00<?, ?it/s]

Converting data for SI
Converting data for DE


 15%|█▌        | 6/39 [00:00<00:02, 12.79it/s]

Converting data for IS
Converting data for XK
Converting data for CH
Converting data for AT
Converting data for DK


 26%|██▌       | 10/39 [00:00<00:01, 18.80it/s]

...

Converting data for HU
Converting data for GR


 97%|█████████▋| 38/39 [00:04<00:00,  8.89it/s]

Converting data for EE
Converting data for FI


100%|██████████| 39/39 [00:04<00:00,  8.83it/s]


,Samplingpoint,PollutantCode,PollutantName,Month,Latitude,Longitude,Value,Unit,Country
0,SPO-SI0001A_00001_100,1,SO2,2021-01-01,46.23451,15.26248,3.079530,ug.m-3,Slovenia
1,SPO-SI0001A_00001_100,1,SO2,2021-02-01,46.23451,15.26248,4.090818,ug.m-3,Slovenia
2,SPO-SI0001A_00001_100,1,SO2,2021-03-01,46.23451,15.26248,4.257419,ug.m-3,Slovenia
3,SPO-SI0001A_00001_100,1,SO2,2021-04-01,46.23451,15.26248,2.892125,ug.m-3,Slovenia
4,SPO-SI0001A_00001_100,1,SO2,2021-05-01,46.23451,15.26248,2.136048,ug.m-3,Slovenia


In [41]:
client = airbase.AirbaseClient()
countries = client.countries
print(set(countries))

{'SI', 'DE', 'IS', 'XK', 'CH', 'AT', 'DK', 'AL', 'HR', 'LU', 'SK', 'PL', 'GB', 'IE', 'BA', 'BE', 'TR', 'LT', 'IT', 'ES', 'MT', 'RO', 'BG', 'RS', 'NL', 'CZ', 'PT', 'MK', 'SE', 'ME', 'FR', 'NO', 'LV', 'AD', 'CY', 'HU', 'GR', 'EE', 'FI'}


In [11]:
df = pd.read_csv(f"../data/aggregated_air_quality/IT.csv", sep=",")

# print just 2014 values
df_2014 = df[(df['Date'] < '2015-01-01') & (df['Date'] >= '2014-01-01') & (df['PollutantCode'] == 7)]
df_2014.head()

,Samplingpoint,PollutantCode,Date,Latitude,Longitude,Value,Unit,PollutantName
49515,SPO.IT0448A_7_UV-P_1994-01-01_00:00:00,7,2014-01-01,45.42944,12.31389,12.869565,ug.m-3,O3
49516,SPO.IT0448A_7_UV-P_1994-01-01_00:00:00,7,2014-01-02,45.42944,12.31389,6.826087,ug.m-3,O3
49517,SPO.IT0448A_7_UV-P_1994-01-01_00:00:00,7,2014-01-03,45.42944,12.31389,5.347826,ug.m-3,O3
49518,SPO.IT0448A_7_UV-P_1994-01-01_00:00:00,7,2014-01-04,45.42944,12.31389,12.000000,ug.m-3,O3
49519,SPO.IT0448A_7_UV-P_1994-01-01_00:00:00,7,2014-01-05,45.42944,12.31389,15.217391,ug.m-3,O3


In [8]:
# Done: AD - AL - BA
# Skipped: AT - BE
df = aggregate_data('BG')

Reading data for BG
Data read.
Start length: 7994407 - End length: 460450


In [ ]:
for country in countries:
    try:
        df = aggregate_data(country)
    except Exception as e:
        print(f"Error processing {country}: {e}")
        continue
    print(f"Aggregated data for {country} saved.")

Reading data for AD
Data read.
Start length: 1307509 - End length: 50601
Aggregated data for AD saved.
Reading data for AL
Data read.
Start length: 2322222 - End length: 96261
Aggregated data for AL saved.
Reading data for AT


In [7]:
import pandas as pd
import plotly.express as px

def draw_average_value_map(df: pd.DataFrame, output_html="air_quality_map.html"):
    """
    Draws a beautiful, web-optimized scatter map showing average 'Value' at each location.
    Intended for use inside a webpage (HTML export, polished layout).

    Args:
        df (pd.DataFrame): Must contain 'Latitude', 'Longitude', and 'Value' columns.
        output_html (str): Path to save the generated HTML.

    Returns:
        None
    """
    # Aggregate values by location
    grouped = df.groupby(['Latitude', 'Longitude'], as_index=False)['Value'].mean()

    # Create scatter map with polished visual style
    fig = px.scatter_mapbox(
        grouped,
        lat='Latitude',
        lon='Longitude',
        color='Value',
        color_continuous_scale='Turbo',  # Vibrant but smooth gradient
        size_max=12,
        zoom=3.5,
        mapbox_style="carto-positron",  # Clean, neutral for web
        title="Air Quality Index in Europe",
        height=600,
    )

    # Clean and centered layout
    fig.update_layout(
        title=dict(
            text="Air Quality Index in Europe",
            x=0.5,
            xanchor='center',
            font=dict(size=22, family="Helvetica, Arial, sans-serif", color="#333")
        ),
        coloraxis_colorbar=dict(
            title="AQI",
            title_side="right",
            ticks="outside",
            tickfont=dict(size=12),
            #titlefont=dict(size=14),
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
    )

    # Export to standalone HTML (great for embedding in iframes)
    fig.write_html(output_html, include_plotlyjs="cdn", full_html=False)

In [8]:
draw_average_value_map(df, output_html="heatmap.html")

/tmp/ipykernel_356092/52510619.py:20: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


In [4]:
df = pd.read_json(f"../data/aggregated_air_quality_monthly_json/all.json", lines=True)
print(df.head())

           Samplingpoint  PollutantCode PollutantName       Month  Latitude  \
0  SPO-SI0001A_00001_100              1           SO2  2021-01-01  46.23451   
1  SPO-SI0001A_00001_100              1           SO2  2021-02-01  46.23451   
2  SPO-SI0001A_00001_100              1           SO2  2021-03-01  46.23451   
3  SPO-SI0001A_00001_100              1           SO2  2021-04-01  46.23451   
4  SPO-SI0001A_00001_100              1           SO2  2021-05-01  46.23451   

   Longitude     Value    Unit   Country  
0   15.26248  3.079530  ug.m-3  Slovenia  
1   15.26248  4.090818  ug.m-3  Slovenia  
2   15.26248  4.257419  ug.m-3  Slovenia  
3   15.26248  2.892125  ug.m-3  Slovenia  
4   15.26248  2.136048  ug.m-3  Slovenia  


In [11]:
import pandas as pd
import json

df['Month'] = pd.to_datetime(df['Month'])
df['Year'] = df['Month'].dt.year

grouped = df.groupby(['Year', 'Samplingpoint', 'PollutantName', 'Country'], as_index=False).agg(
    avg_value=('Value', 'mean'),
    Unit=('Unit', 'first'),
    Latitude=('Latitude', 'first'),
    Longitude=('Longitude', 'first')
)

display(grouped.head())

grouped['avg_value'] = grouped['avg_value'].round(4)
result = {}

for _, row in grouped.iterrows():
    year = str(row['Year'])
    pollutant = row['PollutantName']
    samplingpoint = row['Samplingpoint']
    country = row['Country']
    value = row['avg_value']
    unit = row['Unit']
    longitude = row['Longitude']
    latitude = row['Latitude']

    if year not in result:
        result[year] = {}
    if pollutant not in result[year]:
        result[year][pollutant] = {}
    if country not in result[year][pollutant]:
        result[year][pollutant][country] = []

    result[year][pollutant][country].append({
        "Country": country,
        "Samplingpoint": samplingpoint, 
        "Concentration": value,
        "Unit": unit,
        "Longitude": longitude,
        "Latitude": latitude,
    })

with open("../data/aggregated_air_quality_yearly_json/all.json", "w") as f:
    json.dump(result, f, indent=2)


,Year,Samplingpoint,PollutantName,Country,avg_value,Unit,Latitude,Longitude
0,1990,SPO.04.S156.3945.1.1,SO2,Austria,5.385018,ug.m-3,48.25747,13.03923
1,1990,SPO.04.S156.3950.7.1,O3,Austria,0.000000,ug.m-3,48.25747,13.03923
2,1990,SPO.04.S173.56394.8.1,NO2,Austria,42.835243,ug.m-3,48.27975,14.36650
3,1990,SPO.04.S173.56395.1.1,SO2,Austria,11.854724,ug.m-3,48.27975,14.36650
4,1990,SPO.06.119.4942.1.1,SO2,Austria,18.019247,ug.m-3,47.21037,14.82528


In [ ]:
import json
import pandas as pd

# Load the nested JSON
with open("../data/aggregated_air_quality_yearly_json/all.json", "r") as f:
    data = json.load(f)

# Flatten it manually
rows = []
for year, year_data in data.items():
    for pollutant, pollutant_data in year_data.items():
        for country, records in pollutant_data.items():
            for record in records:
                flat_record = {
                    "Year": int(year),
                    "Pollutant": pollutant,
                    "Country": country,
                    **record  # includes Samplingpoint, Concentration, Latitude, Longitude, etc.
                }
                rows.append(flat_record)

# Create DataFrame
df = pd.DataFrame(rows)

# Optional: inspect
display(df.head())

,Year,Pollutant,Country,Samplingpoint,Concentration,Unit,Longitude,Latitude
0,1990,SO2,Austria,SPO.04.S156.3945.1.1,5.3850,ug.m-3,13.03923,48.25747
1,1990,SO2,Austria,SPO.04.S173.56395.1.1,11.8547,ug.m-3,14.36650,48.27975
2,1990,SO2,Austria,SPO.06.119.4942.1.1,18.0192,ug.m-3,14.82528,47.21037
3,1990,SO2,Austria,SPO.06.156.1813.1.1,5.8418,ug.m-3,15.88222,47.34806
4,1990,SO2,Austria,SPO.06.170.5937.1.1,16.3170,ug.m-3,15.43308,47.04172
